In [72]:
%load_ext autoreload
%autoreload 2
import importlib
import sys

import numpy as np
import matplotlib.pyplot as plt
import pyomo.environ as pyo
from mpl_toolkits import mplot3d
from useful import get_data, SVM_linear, SVM_nonlinear, get_features, print_learned_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
importlib.reload(sys.modules[print_learned_model.__module__])
importlib.reload(sys.modules[get_features.__module__])

<module 'useful' from '/Users/Home/Documents/Coding/Arduino/gesture_recogniser_train_model/useful.py'>

In [74]:
up_data, up_time = get_data("up_data.txt")
circle_data, circle_time = get_data("circle_data.txt")
left_data, left_time = get_data("left_data.txt")
data = np.array([*up_data, *circle_data, *left_data]).T
time_array = np.array([*up_time, *circle_time, *left_time]).T

(d, N) = time_array.shape
N_h = N//3

In [75]:
# Condense and format feature data
X = get_features(data, time_array)
up_t = np.array([*[-1]*N_h, *[1]*N_h, *[1]*N_h])
circle_t = np.array([*[1]*N_h, *[-1]*N_h, *[1]*N_h])
left_t = np.array([*[1]*N_h, *[1]*N_h, *[-1]*N_h])
print(X.shape)

V_range =  [[-0.31144, 0.38423999999999986], [-0.21687999999999996, 0.35688000000000003], [0.01208, 1.6137850000000007]]
Pos_range =  [[-0.27893647249999987, 0.2806080000000001], [-0.18462015999999992, 0.35701439999999995], [0.00031872, 1.3821549949999998]]
(300, 39)


In [76]:
# Learn SVM
C, gam = 10, 0.8

# Set kernel functions
K = lambda Xn, Xm, gam=gam: pyo.exp(-gam * np.linalg.norm(Xn - Xm)**2)
K_pred = lambda Xn, Xm, gam=gam: np.exp(-gam * np.linalg.norm(Xn - Xm, axis=1)**2)
up_a, up_b, up_svi = SVM_nonlinear(X, up_t, K, C=C)
circle_a, circle_b, circle_svi = SVM_nonlinear(X, circle_t, K, C=C)
left_a, left_b, left_svi = SVM_nonlinear(X, left_t, K, C=C)

In [77]:
def test_nonlinear_SVM(X_test, test_t, X, t, a, b):
    (N_test, d) = X_test.shape
    N = X.shape[0]
    y = np.zeros(N_test)
    for n in range(N):
        y += a[n] * t[n] * K_pred(X[n], X_test)
        print("This is the value: ", K_pred(X[n], X_test))
    errors = [1 if y[i]*test_t[i] < 0 else 0 for i in range(N_test)]
    print("Samples Tested: ", N_test)
    print("Total Errors: ", sum(errors))
    print("Accuracy: ", (N - sum(errors))/N)

In [78]:
print("Up Test: ")
test_nonlinear_SVM(X, up_t, X, up_t, up_a, up_b)
print("\nCircle Test: " )
test_nonlinear_SVM(X, circle_t, X, circle_t, circle_a, circle_b)
print("\nLeft Test: " )
test_nonlinear_SVM(X, left_t, X, left_t, left_a, left_b)

Up Test: 
Samples Tested:  300
Total Errors:  0
Accuracy:  1.0

Circle Test: 
Samples Tested:  300
Total Errors:  0
Accuracy:  1.0

Left Test: 
Samples Tested:  300
Total Errors:  0
Accuracy:  1.0


In [79]:
# Get test data
up_test_data, up_time = get_data("up_test_data.txt")
circle_test_data, circle_time = get_data("circle_test_data.txt")
left_test_data, left_time = get_data("left_test_data.txt")
test_data = np.array([*up_test_data, *circle_test_data, *left_test_data]).T
test_time_array = np.array([*up_time, *circle_time, *left_time]).T

(test_d, test_N) = test_time_array.shape
test_N_h = test_N//3

X_test = get_features(test_data, test_time_array)
test_up_t = np.array([*[-1]*test_N_h, *[1]*test_N_h, *[1]*test_N_h])
test_circle_t = np.array([*[1]*test_N_h, *[-1]*test_N_h, *[1]*test_N_h])
test_left_t = np.array([*[1]*test_N_h, *[1]*test_N_h, *[-1]*test_N_h])

V_range =  [[-0.26947, 0.32308000000000003], [-0.18961500000000003, 0.452545], [0.01168, 1.60166]]
Pos_range =  [[-0.2650955, 0.29902286749999984], [-0.09181547999999994, 0.4705875325], [0.0003113600000000001, 1.3665027049999998]]


In [80]:
print("Up Test: ")
test_nonlinear_SVM(X_test, test_up_t, X, up_t, up_a, up_b)
print("\nCircle Test: " )
test_nonlinear_SVM(X_test, test_circle_t, X, circle_t, circle_a, circle_b)
print("\nLeft Test: " )
test_nonlinear_SVM(X_test, test_left_t, X, left_t, left_a, left_b)

Up Test: 
Samples Tested:  225
Total Errors:  0
Accuracy:  1.0

Circle Test: 
Samples Tested:  225
Total Errors:  0
Accuracy:  1.0

Left Test: 
Samples Tested:  225
Total Errors:  0
Accuracy:  1.0


In [81]:
print(left_a)

[0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666664 0.66666372 0.66666377
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669
 0.66666669 0.66666669 0.66666669 0.66666669 0.66666669 0.6666

In [82]:
print_learned_model(X, up_t, up_a, up_b, circle_t, circle_a, circle_b, left_t, left_a, left_b)

extern float X[300][39] = {
	{-5.0600000000000005, -4.79, 89.08999999999999, -6.831679999999999, -3.8083199999999975, 71.31232000000003, -5.4140108799999975, -1.6692985599999988, 39.974878079999996, 11.0, 57.0, 32.0, 0.0, 0.0, 20.0, 80.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 56.0, 44.0, 0.0, 0.0, 44.0, 56.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}, 
	{-2.5500000000000003, -6.259999999999999, 90.40999999999998, -3.08632, -3.786000000000001, 72.29623999999998, -3.3439366399999995, -1.5065503999999998, 40.33723391999999, 0.0, 36.0, 64.0, 0.0, 0.0, 12.0, 70.0, 18.0, 0.0, 0.0, 19.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0}, 
	{-3.47, -6.25, 89.16000000000001, -5.042479999999999, -4.424880000000001, 71.40903999999998, -4.878581120000002, -2.2005587199999996, 39.997359359999976, 5.0, 47.0, 48.0, 0.0, 0.0, 18.0, 82.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 59.0, 41.0, 0.0, 0.0, 6.0, 94.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,